In [ ]:
#---------------------------
#--- Paquetes
#---------------------------
import pandas as pd
import seaborn as sns

'''Permite buscar y recuperar una lista de nombres de archivos que coinciden con un patrón específico de nombre
de archivo en un directorio o en una jerarquía de directorios.'''

import glob

import numpy as np
import os
from datetime import datetime, timedelta

import matplotlib.pyplot as plt


In [ ]:
#-----------------------------
#--- Sales
#----------------------------

'''archivo csv con la inf de venta historica
este archivo csv es el resultado de unificar todos los xlsx de sharepoint, filtrando las columnas de interes para el modelo'''

ruta=r'/home/sebastian/Documentos/programas/Forescast Work/Sales_History.csv'
df_SalesAndProduct=pd.read_csv(ruta,index_col=None, header=0,dtype=str)

In [ ]:
'''se leen todos los xlsx de sharepoint, se unifican en un solo df_sales, 
se lee el xlsx de master products
se cruza df_sales con df_products para obtener la clasificacion de los sku con venta'''


#--------------------
# --- SALES 
#--------------------

'''Ruta donde se encuentre la inf de venta'''
paht_data_historica=r'C:\Users\SSN0609\OneDrive - Stanley Black & Decker\Dashboard Marketing\Sales'
#Crea una lst con la ubicacion de los archivos
all_files_sales = glob.glob(paht_data_historica + "/*Sales*.xlsx")
#Filtra los files que son de venta
filtered_files_sales = [file for file in all_files_sales if "path" not in file]

lst_sales=[]
'''Recorre la lista de ubicaciones y lee el xlsx'''
for filename in filtered_files_sales:
    df = pd.read_excel(filename, index_col=None, header=0,dtype=str)
    lst_sales.append(df)
len(lst_sales)

'''Concatena la info de lst_sales(col: inf xlsx) de manera vertical'''
df_sales= pd.concat(lst_sales,axis=0,ignore_index=True)

#--------------------
# --- PRODUCTS 
#--------------------
paht_data_product=r'C:\Users\SSN0609\OneDrive - Stanley Black & Decker\Master Data\Master Data Products Reclasif.xlsx'
df_products=pd.read_excel(paht_data_product,index_col=None,header=0,dtype=str)

#-------------------------------
# --- Eleccion Columnas
#-------------------------------

Columns_products=['SKU', 'SKU Base', 'SKU Description', 'Brand ', 'GPP Division Code',
       'GPP Division Reclasif', 'GPP Category Reclasif', 'GPP Portfolio ',
       'SBU', 'Super SBU', 'GPP SBU', 'GPP SBU Description']
df_products=df_products[Columns_products]

columns_sales=['Fiscal Year', 'Fiscal Period', 'Country Code', 
       'Sold-To Customer Code','Country Material', 'Total Sales', 'Total Cost', 'Units Sold']
df_sales=df_sales[columns_sales]

#-----------------------------
#--- Merge
#-----------------------------

df_SalesAndProduct=pd.merge(
    df_sales,df_products,how='left',left_on='Country Material',right_on='SKU'
)

print(f'len(df_sales): {len(df_sales)}')
print(f'len(df_salesProdu): {len(df_SalesAndProduct)}')

#-------------------------------
#---- Date
#-------------------------------
dict_month={
        'jan':'01',
        'feb':'02',
        'mar':'03',
        'apr':'04',
        'may':'05',
        'jun':'06',
        'jul':'07',
        'aug':'08',
        'sep':'09',
        'oct':'10',
        'nov':'11',
        'dec':'12',
        '1':'01'
    }
def mes(row):
    month = row['Fiscal Period'].lower()
    month_num = dict_month[month]
    year = row['Fiscal Year']
    date_str = month_num + '-' + year
    return date_str

df_SalesAndProduct['Date']=df_SalesAndProduct.apply(mes,axis=1) 
df_SalesAndProduct['Date']=pd.to_datetime(df_SalesAndProduct['Date'],format='%m-%Y')
df_SalesAndProduct.head()

for col in df_SalesAndProduct.columns:
    df_SalesAndProduct.rename(columns={col: col.strip()}, inplace=True)


In [20]:
df_SalesAndProduct.head(-50)

,Fiscal Year,Fiscal Period,Country Code,Sold-To Customer Code,Country Material,Total Sales,Total Cost,Units Sold,SKU,SKU Base,...,Brand,GPP Division Code,GPP Division Reclasif,GPP Category Reclasif,GPP Portfolio,SBU,Super SBU,GPP SBU,GPP SBU Description,Date
0,2022,Apr,Argentina,AR/00005365,ST5010,11.33,1.55,1,ST5010,ST5010,...,Stanley,30,ACCESSORIES (PTA),DRILLING - METAL,ROUND SHANK METAL DRILLING,PTA,HTAS,PTA,POWER TOOL ACCESSORIES,2022-04-01
1,2022,Apr,Argentina,AR/00000141,71-727,949.84,281.05,150,71-727,71-727,...,Black + Decker,30,ACCESSORIES (PTA),LINEAR EDGE - RECIP,ALLIGATOR BLADES,PTA,HTAS,PTA,POWER TOOL ACCESSORIES,2022-04-01
2,2022,Apr,Argentina,AR/00000141,77-717 PS,261.07,151.83,33,77-717 PS,77-717,...,Black + Decker,30,ACCESSORIES (PTA),LINEAR EDGE - RECIP,ALLIGATOR BLADES,PTA,HTAS,PTA,POWER TOOL ACCESSORIES,2022-04-01
3,2022,Apr,Argentina,AR/00000141,77-757 PS,845.57,257.51,84,77-757 PS,77-757,...,Black + Decker,30,ACCESSORIES (PTA),LINEAR EDGE - RECIP,ALLIGATOR BLADES,PTA,HTAS,PTA,POWER TOOL ACCESSORIES,2022-04-01
4,2022,Apr,Argentina,AR/00000141,77-770 PS,126.04,55.51,6,77-770 PS,77-770,...,Black + Decker,30,ACCESSORIES (PTA),CIRCULAR SAW BLADES,WOOD CUTTING,PTA,HTAS,PTA,POWER TOOL ACCESSORIES,2022-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6600292,2024,Sep,Mexico,MX/C21500,PSD,3189.2,1209.09,492,PSD,0,...,Other,70,PS PARTS (PSD),PS PARTS (PSD),PS PARTS,PSD,PSD,PSD,PRODUCT SERVICE DIVISION,2024-09-01
6600293,2024,Sep,Mexico,MX/C21520,PSD,713.26,180.9,161,PSD,0,...,Other,70,PS PARTS (PSD),PS PARTS (PSD),PS PARTS,PSD,PSD,PSD,PRODUCT SERVICE DIVISION,2024-09-01
6600294,2024,Sep,Mexico,MX/C21529,PSD,3899.61,1394.61,1315,PSD,0,...,Other,70,PS PARTS (PSD),PS PARTS (PSD),PS PARTS,PSD,PSD,PSD,PRODUCT SERVICE DIVISION,2024-09-01
6600295,2024,Sep,Mexico,MX/C21668,PSD,1502.53,455.35,453,PSD,0,...,Other,70,PS PARTS (PSD),PS PARTS (PSD),PS PARTS,PSD,PSD,PSD,PRODUCT SERVICE DIVISION,2024-09-01


In [14]:
'''Evaluación del Enfoque y Sugerencias

    Función de Filtrado para Generar Series de Tiempo:
        Comentario: Crear una función que permita generar series de tiempo para un parámetro específico es una excelente idea. Esto te permitirá ver la evolución de las ventas desde perspectivas clave como marca, país o SBU, sin tener que filtrar manualmente.
        Sugerencia de Mejora: Podrías optimizar la función para permitir múltiples filtros, por ejemplo, que acepte un diccionario con varias condiciones ({"brand": "dewalt", "country": "US"}) para flexibilizar los análisis.
        Bonus: Agregar visualizaciones automáticas dentro de la función, como gráficos de líneas, descomposiciones de tendencia, o correlogramas, te dará una visión rápida del comportamiento de cada serie.

    Función para Extraer Métricas Clave de la Serie de Tiempo:
        Comentario: La función que describe los valores importantes (como tendencia, estacionalidad, varianza, autocorrelación, etc.) es fundamental para seleccionar el modelo adecuado.
        Sugerencia de Mejora: Además de devolver métricas clave, podrías considerar agregar un análisis exploratorio automatizado que incluya pruebas estadísticas, como el test de Dickey-Fuller para estacionalidad, y métricas de variabilidad estacional para facilitar la comparación de series entre categorías.
        Bonus: Incluir una clasificación preliminar en función de estas métricas para sugerir directamente un modelo (por ejemplo, si la estacionalidad es fuerte, sugerir SARIMA).

    Función de Modelado Basado en Métricas de la Serie de Tiempo:
        Comentario: Automatizar el ajuste del modelo con base en las características de cada serie de tiempo es un excelente enfoque.
        Sugerencia de Mejora: Considera que, además de seleccionar un modelo, podrías usar una estructura de prueba de múltiples modelos (modelo base + modelo estacional) en paralelo y luego elegir el modelo con menor error de validación.
        Bonus: Además de SARIMA o Prophet, podrías incluir modelos alternativos como TBATS o modelos híbridos (SARIMA + XGBoost) en función de las características, especialmente si tienes datos altamente no lineales.

    Función para Almacenar Predicciones en un DataFrame de Resultados:
        Comentario: La idea de almacenar las predicciones de cada serie de tiempo es excelente para poder consultar los resultados de manera centralizada.
        Sugerencia de Mejora: Podrías organizar el DataFrame para tener un índice multinivel con parámetros como Fiscal Period, Country, Brand, SBU, etc., lo que permitirá filtrar y resumir los resultados fácilmente.
        Bonus: Añadir una columna que indique el error de predicción del modelo para cada serie en el DataFrame de resultados, lo cual permitirá monitorear la precisión de los modelos en el tiempo.

    Función para Generar Matrices Estructuradas (por período fiscal, país, marca, etc.):
        Comentario: Generar matrices estructuradas por características es útil para visualizar los patrones de ventas y, potencialmente, crear correlaciones o relaciones entre variables.
        Sugerencia de Mejora: Asegúrate de que las matrices de datos tengan una estructura consistente (llenando los datos faltantes si es necesario) y considera una agregación preliminar en cada período. Podrías también aprovechar estas matrices para detectar correlaciones interanuales o estacionales entre países y marcas.
        Bonus: En lugar de solo generar la matriz de datos, podrías considerar crear matrices de coeficientes de correlación dinámicas que puedan usarse como input para ajustar modelos combinados.

    Función para el Análisis de Correlación entre Variables:
        Comentario: Evaluar la correlación entre las variables y promediar modelos para las variables altamente correlacionadas es una estrategia sólida para evitar duplicidad en los resultados.
        Sugerencia de Mejora: Podrías definir un umbral de correlación para decidir qué variables se consideran "altamente correlacionadas" y asegurarte de aplicar el promedio de modelos solo en aquellas variables donde la correlación supera este umbral.
        Bonus: En lugar de promediar modelos, podrías optar por modelos de ensamble (ensemble models) donde los modelos de las variables correlacionadas aporten predicciones ponderadas, maximizando la precisión para categorías similares.
'''


'Evaluación del Enfoque y Sugerencias\n\n    Función de Filtrado para Generar Series de Tiempo:\n        Comentario: Crear una función que permita generar series de tiempo para un parámetro específico es una excelente idea. Esto te permitirá ver la evolución de las ventas desde perspectivas clave como marca, país o SBU, sin tener que filtrar manualmente.\n        Sugerencia de Mejora: Podrías optimizar la función para permitir múltiples filtros, por ejemplo, que acepte un diccionario con varias condiciones ({"brand": "dewalt", "country": "US"}) para flexibilizar los análisis.\n        Bonus: Agregar visualizaciones automáticas dentro de la función, como gráficos de líneas, descomposiciones de tendencia, o correlogramas, te dará una visión rápida del comportamiento de cada serie.\n\n    Función para Extraer Métricas Clave de la Serie de Tiempo:\n        Comentario: La función que describe los valores importantes (como tendencia, estacionalidad, varianza, autocorrelación, etc.) es fundam

In [ ]:
df_SalesAndProduct.columns
df

Index(['Fiscal Year', 'Fiscal Period', 'Country Code', 'Sold-To Customer Code',
       'Country Material', 'Total Sales', 'Total Cost', 'Units Sold', 'SKU',
       'SKU Base', 'SKU Description', 'Brand ', 'GPP Division Code',
       'GPP Division Reclasif', 'GPP Category Reclasif', 'GPP Portfolio ',
       'SBU', 'Super SBU', 'GPP SBU', 'GPP SBU Description', 'Date'],
      dtype='object')

In [70]:
df_SalesAndProduct['Date']=pd.to_datetime(df_SalesAndProduct['Date'],format='%Y-%m-%d')
for col in df_SalesAndProduct.columns:
    df_SalesAndProduct.rename(columns={col: col.strip()}, inplace=True)

df_SalesAndProduct = df_SalesAndProduct.sort_values(by=['Date', 'Country Code', 'Brand'], ascending=[True, False, False])
df_SalesAndProduct['Total Sales']=pd.to_numeric(df_SalesAndProduct['Total Sales'])

In [86]:
print(f' numer total de registros de ventas:{len(df_SalesAndProduct)}')
print(f"numero total de devoluciones: {len(df_SalesAndProduct[df_SalesAndProduct['Total Sales'] < 0])}")
print(f"% devoluciones:{len(df_SalesAndProduct)/len(df_SalesAndProduct[df_SalesAndProduct['Total Sales'] < 0])}")


 numer total de registros de ventas:6600347
numero total de devoluciones: 239202
% devoluciones:27.593193200725747


In [79]:
#---------------------------------------
#---- Creacion series de tiempo
#---------------------------------------


def generar_serie_tiempo(df, filtros):
    """
    Filtra los datos de ventas según los filtros especificados y devuelve una serie de tiempo.
    Args:
        df (pd.DataFrame): DataFrame con datos de ventas.
        filtros (dict): Diccionario de condiciones para filtrar.
    Returns:
        pd.Series: Serie de tiempo de ventas filtrada.
    """
    for clave, valor in filtros.items():
        df = df[df[clave] == valor]
    serie = df.set_index('Date')['Total Sales']
    return serie

In [73]:
filtros={
'Brand':[''],
'Country Code':[''],
'GPP Division Code':[]

}
serie=generar_serie_tiempo(df_SalesAndProduct,{'Brand':'Dewalt','Country Code':'Mexico'})
serie

Date
2022-01-01     5.41
2022-01-01     3.51
2022-01-01    52.06
2022-01-01     3.75
2022-01-01     2.68
              ...  
2024-10-01    10.96
2024-10-01     3.65
2024-10-01     2.44
2024-10-01    13.61
2024-10-01    18.15
Name: Total Sales, Length: 3941, dtype: float64

In [78]:
len(serie[serie< 0])

14

In [64]:
import statsmodels.api as sm

def extraer_metrica_serie_tiempo(serie):
    descomposicionAdd = sm.tsa.seasonal_decompose(serie, model='additive', period=12)
    log_ventas = np.log(serie["Total Sales"])
    descomposicionMul = sm.tsa.seasonal_decompose(log_ventas, model='multiplicative', period=12)

    return {
        "Mod Add tendencia": descomposicionAdd.trend.mean(),
        "Mod Add estacionalidad": descomposicionAdd.seasonal.mean(),
        "Mod Add ruido": descomposicionAdd.resid.mean(),
        
        "Mod Mul tendencia": descomposicionMul.trend.mean(),
        "Mod Mul estacionalidad": descomposicionMul.seasonal.mean(),
        "Mod Mul ruido": descomposicionMul.resid.mean(),
        
        "ADF Test": sm.tsa.adfuller(serie)[1]    # p-value
    }

In [65]:
extraer_metrica_serie_tiempo(serie)

KeyError: 'Total Sales'